© 2024 Rebecca J. Rousseau and Justin B. Kinney, *Algebraic and diagrammatic methods for the rule-based modeling of multi-particle complexes*. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).
___

# Deterministic simulation for homopolymer system

In [1]:
import numpy as np
from itertools import combinations, permutations
from scipy import sparse, linalg

In [2]:
# Define "dictionary" of mathematical operators tracking field formation (hat), degradation (check), presence (bar),
# and absence (tilde), as well as identity (id) and "zero" (zero) operators.

mat_dict = dict(
    hat=sparse.csr_matrix(np.array([[0, 1], [0, 0]])),
    bar=sparse.csr_matrix(np.array([[1, 0], [0, 0]])),
    tilde=sparse.csr_matrix(np.array([[0, 0], [0, 1]])),
    check=sparse.csr_matrix(np.array([[0, 0], [1, 0]])),
    id=sparse.csr_matrix(np.array([[1, 0], [0, 1]])),
    zero=sparse.csr_matrix(np.array([[0, 0], [0, 0]]))
)

In [3]:
def multikron(mat_list):
    """Computes Kronecker product of multiple matrices"""
    n = len(mat_list)
    assert n>0
    out_mat = mat_list[0]
    for mat in mat_list[1:]:
        out_mat = sparse.kron(out_mat, mat, format='csc')
    return out_mat

In [4]:
N = 3 # Total number of possible monomer internal states
K = int(N + 2*N + N**2) # Total number of mode operator fields
r_A_cre = 2 # Rate of monomer creation
r_A_deg = 2 # Rate of monomer degradation
r_I_cre = 0.5 # Rate of bond formation
r_I_deg = 1 # Rate of bond degradation

In [5]:
# A_i: k = 1 .. N (monomer mode operators)
# a_i: k = N+1 .. 2N (monomer binding site mode operators)
# b_i: k = 2N+1 .. 3N (monomer binding site mode operators)
# I_ij: k = 3N+1 .. 3N+N(N-1)/2 (bond mode operators)

### Create single-digit index for each bond type:

# Define all possible monomer index bond pairs

ij_list = [(i,j) for i in range(N) for j in range(N)]

# Assign index bond pair to a single-digit index label

dk_to_ij_dict = dict([(dk, ij) for dk, ij in enumerate(ij_list)])

ij_to_dk_dict = dict(zip(dk_to_ij_dict.values(), dk_to_ij_dict.keys()))

In [6]:
def op(field_name, field_type, field_index):
    if field_name=='A':
        i = field_index
        k = i
    
    elif field_name=='a':
        i = field_index
        k = N+i
        
    elif field_name=='b':
        i = field_index
        k = 2*N+i
        
    elif field_name=='I':
        i,j = field_index
        k = 3*N + ij_to_dk_dict[(i,j)]
        
    return multikron([mat_dict[field_type] if l==k else mat_dict['id'] for l in range(K)])

In [7]:
# Compute transition operator W

W = multikron([mat_dict['zero']]*K)
for i in range(N):
    W += r_A_cre*(op('A','hat',i) - op('A','tilde',i))*op('a','tilde',i)*op('b','tilde',i)*op('I','tilde',(i,i)) + \
         r_A_deg*(op('A','check',i) - op('A','bar',i))*op('a','tilde',i)*op('b','tilde',i)*op('I','tilde',(i,i))
for i in range(N):
    for j in range(N):
        W += r_I_cre*op('A','bar',i)*op('A','bar',j)*(op('a','hat',i)*op('b','hat',j)*op('I','hat',(i,j))-\
                      op('a','tilde',i)*op('b','tilde',j)*op('I','tilde',(i,j))) +\
             r_I_deg*op('A','bar',i)*op('A','bar',j)*(op('a','check',i)*op('b','check',j)*op('I','check',(i,j)) -\
                      op('a','bar',i)*op('b','bar',j)*op('I','bar',(i,j)))

In [8]:
# Compute A counting matrix

A_bar = multikron([mat_dict['zero']]*K)
for i in range(N):
    A_bar += op('A','bar',i)*op('a','tilde',i)*op('b','tilde',i)*op('I','tilde',(i,i))  # count FREE monomers

In [9]:
# Compute 2-chain counting matrix

nn = 2

two_chain_bar = multikron([mat_dict['zero']]*K)
for k in range(len(ij_list)):
    if ij_list[k][0]!=ij_list[k][1]:
        Ipart_bar2c = multikron([mat_dict['id']]*K)
        Ipart_bar2c *= op('A','bar',ij_list[k][0])*op('A','bar',ij_list[k][1])*op('I','bar',ij_list[k])*op('a','bar',ij_list[k][0])*op('b','bar',ij_list[k][1])*op('a','tilde',ij_list[k][1])*op('b','tilde',ij_list[k][0])
        two_chain_bar += Ipart_bar2c

In [11]:
# Compute 3-chain counting matrix

nn = 3
mon_list = [i for i in range(N)]
comb = list(permutations(mon_list,nn))
    
three_chain_bar = multikron([mat_dict['zero']]*K)
for k in range(len(comb)):
    Ipart_bar3c = multikron([mat_dict['id']]*K)
    Ipart_bar3c *= op('A','bar',comb[k][0])*op('A','bar',comb[k][1])*op('A','bar',comb[k][2])*op('I','bar',(comb[k][0],comb[k][1]))*op('I','bar',(comb[k][1],comb[k][2]))*op('a','bar',comb[k][0])*op('b','bar',comb[k][1])*op('a','bar',comb[k][1])*op('b','bar',comb[k][2])*op('a','tilde',comb[k][2])*op('b','tilde',comb[k][0])
    three_chain_bar += Ipart_bar3c

In [12]:
# Compute 1-ring counting matrix

A_ring_bar = multikron([mat_dict['zero']]*K)
for i in range(N):
    A_ring_bar += op('I','bar',(i,i))

In [13]:
# Compute 2-ring counting matrix

nn = 2
two_ring_bar = multikron([mat_dict['zero']]*K)
for k in range(len(ij_list)):
    if ij_list[k][0] < ij_list[k][1]:
        Ipart_bar2r = multikron([mat_dict['id']]*K)
        Ipart_bar2r = op('A','bar',comb[k][0])*op('A','bar',comb[k][1])*op('I','bar',ij_list[k])*op('I','bar',(ij_list[k][1],ij_list[k][0]))
        two_ring_bar += Ipart_bar2r

In [14]:
### Compute n-ring counting matrix

nn = 3

# List all unique sets of 'nn' selected monomers

mon_list2 = [i for i in range(N)]
res2 = list(combinations(mon_list2, nn))

# For a given set, list all possible bond index pairs, then choose 'nn-1' for chain or 'nn' for ring

three_ring_bar = multikron([mat_dict['zero']]*K)
for k in range(len(res2)):
    Ipart_bar3r = multikron([mat_dict['id']]*K)
    ij_bond_list2 = [(i,j) for i in list(res2[k]) for j in list(res2[k]) if j != i]
    res_sub2 = list(combinations(ij_bond_list2,nn))
    for m1 in range(len(list(res2[k]))):
        Ipart_bar3r *= op('A','bar',list(res2[k])[m1])
    for m2 in range(len(res_sub2)):
        a_listring = [res_sub2[m2][t][0] for t in range(nn)]
        b_listring = [res_sub2[m2][tt][1] for tt in range(nn)]
        if ((len(set(a_listring)) == len(a_listring)) and (len(set(b_listring)) == len(b_listring))):
            res_concat2 = ()
            res_concat_unique2 = set()
            for ss in range(nn):
                res_concat2 += res_sub2[m2][ss]
            res_concat_unique2 = set(res_concat2)
            if res_concat_unique2 == set(res2[k]):
                for ss in range(nn):
                    Ipart_bar3r *= op('I','bar',res_sub2[m2][ss])*op('a','bar',res_sub2[m2][ss][0])*op('b','bar',res_sub2[m2][ss][1])
            three_ring_bar += Ipart_bar3r

In [15]:
# Construct ground state vector

ground_el = sparse.csc_matrix(np.array([0, 1]))
ground_state = multikron([ground_el]*K)
ground_state = ground_state.toarray().squeeze()

In [16]:
# Construct sum vector

sum_vec = np.ones(2**K).T

In [17]:
# Compute the number of A after time T

from scipy.sparse.linalg import expm_multiply
t_stop = 10
num_timepoints = 2001
psi_array = expm_multiply(W,ground_state.T,start=0.0, stop=t_stop, num=num_timepoints)

In [18]:
A_of_t = np.zeros(num_timepoints)
Aring_of_t = np.zeros(num_timepoints)
twochain_of_t = np.zeros(num_timepoints)
tworing_of_t = np.zeros(num_timepoints)
threechain_of_t = np.zeros(num_timepoints)
threering_of_t = np.zeros(num_timepoints)
for t in range(num_timepoints):
    psi_t = psi_array[t,:]
    A_of_t[t] = sum_vec.dot(A_bar.dot(psi_t))
    Aring_of_t[t] = sum_vec.dot(A_ring_bar.dot(psi_t))
    twochain_of_t[t] = sum_vec.dot(two_chain_bar.dot(psi_t))
    tworing_of_t[t] = sum_vec.dot(two_ring_bar.dot(psi_t))
    threechain_of_t[t] = sum_vec.dot(three_chain_bar.dot(psi_t))
    threering_of_t[t] = sum_vec.dot(three_ring_bar.dot(psi_t))

In [19]:
folderpath = "../simulationdata"
np.savetxt(f"{folderpath}/A_of_t_homopol_N{N}.csv",A_of_t,delimiter=",")
np.savetxt(f"{folderpath}/Aring_of_t_homopol_N{N}.csv",Aring_of_t,delimiter=",")
np.savetxt(f"{folderpath}/twochain_of_t_homopol_N{N}.csv",twochain_of_t,delimiter=",")
np.savetxt(f"{folderpath}/tworing_of_t_homopol_N{N}.csv",tworing_of_t,delimiter=",")
np.savetxt(f"{folderpath}/threechain_of_t_homopol_N{N}.csv",threechain_of_t,delimiter=",")
np.savetxt(f"{folderpath}/threering_of_t_homopol_N{N}.csv",threering_of_t,delimiter=",")